# Genetic algo for lighthouse sensor distribution on arbitrary mesh

In [1]:
import numpy as np
from stl import mesh as meshstl
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from data.plotmesh import plot_mesh
import math
import random
from pyquaternion import Quaternion
from scipy.linalg import qr
import roslib
import rospy
import math
import tf
rospy.init_node('fixed_tf_broadcaster')

In [2]:
#how many sensors would u like to distribute?
sensorsToDistribute = 25
stl_file = 'roboy_models/TestCube/stls/IcoSphere_360.stl'

In [3]:
#Move Lighthouses to
translationLH1 = [-2.,0,2.]
quat1 = Quaternion(axis=[0,0,1],angle=-np.pi / 2)

global LH1 
LH1 = (translationLH1, quat1)

translationLH2 = [2,0.,2.]
quat2 = Quaternion(axis=[0,0,1], angle=np.pi / 2)

global LH2
LH2 = (translationLH2, quat2)

print(LH1); print(LH2)

([-2.0, 0, 2.0], Quaternion(0.70710678118654757, -0.0, -0.0, -0.70710678118654746))
([2, 0.0, 2.0], Quaternion(0.70710678118654757, 0.0, 0.0, 0.70710678118654746))


In [4]:
from data.rvizMeshVis import meshVisualization

scale = 0.01
position = [0,0,0]
global orientationMesh
orientationMesh = Quaternion(axis=(1,0,0),angle = np.pi*0)

meshVisualization(orientationMesh, stl_file, position, scale)

# Preprocess data

In [5]:
#Get mesh vertices and normals
mesh1 = meshstl.Mesh.from_file(('../'+ stl_file))
#mesh1 = meshstl.Mesh.from_file('../src/roboy_models/roboy_2_0_simplified/meshes/CAD/torso.stl')

global triangles 
global trianglesBackup
triangles = scale * np.matrix(mesh1.points)
trianglesBackup = triangles

vertices = np.reshape(triangles,(len(triangles)*3,3)) 

#Initialize sensors in centers of triangle
sensors = (triangles[:,0:3]+triangles[:,3:6]+triangles[:,6:9])/3

print('%d triangles' %len(triangles))
#print(triangles)
print('')
print('%d vertices' %len(vertices))
#print(vertices)
print('')
print('%d sensors in centers of triangles' %len(sensors))
#print(sensors)
#print(vertices); print(normals); print(sensors)

484 triangles

1452 vertices

484 sensors in centers of triangles


In [6]:
#print(triangles)
#print(triangles.shape)

test = np.concatenate((triangles[:,0:3],triangles[:,3:6],triangles[:,6:9]),axis=1)
#print('test1')
#print(test)
#print(test.shape)


In [7]:
#Prepare vertices, normals and sensor locations for Transformation

verticesRot = np.insert(vertices, 3,values=1,axis=1)
#print(verticesRot)
#print('')

sensorsRot = np.insert(vertices, 3,values=1,axis=1)
#print(sensorsRot)
#print('')

# GA

In [8]:
from deap import algorithms, base, creator, tools

In [9]:
#sensors to dict
global sensor_dict
sensor_dict =  list(zip(range(len(sensors)), sensors.tolist()))

global sensorDictBackup
sensorDictBackup = sensor_dict


In [10]:
from data.rvizSensorVis import sensorVisualization

#color = (r,g,b,a)
sensorVisualization(sensor_dict, rate=500, sphereSize=0.03, color=(0,0,1,1))

In [11]:
creator.create("FitnessMax", base.Fitness, weights=(0.00000001/sensorsToDistribute,1.0,1.0)) # 1 -> maximum probblem
creator.create("Individual", list, fitness=creator.FitnessMax)

In [12]:
toolbox = base.Toolbox()

In [13]:
from data.randomSensor import randomSensor

toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", randomSensor, sensor_dict)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, sensorsToDistribute)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [14]:
toolbox.attr_bool()

170

In [15]:
toolbox.individual()

[267,
 32,
 89,
 192,
 451,
 8,
 385,
 55,
 192,
 382,
 237,
 281,
 356,
 431,
 276,
 132,
 183,
 464,
 478,
 448,
 452,
 479,
 468,
 385,
 65]

# Evaluation (Fitness) Function

In [16]:
from data.RayIntersectsTriangle import rayIntersectsTriangle, rayIntersectsTriangleVisualization

def FitnessFunction(sensors):
    
    br = tf.TransformBroadcaster()
    br.sendTransform((LH1[0][0], LH1[0][1], LH1[0][2]),
                     (quat1[0], quat1[1], quat1[2], quat1[3]),
                     rospy.Time.now(),
                     "lighthouse1",
                     "world")
    br.sendTransform((LH2[0][0], LH2[0][1], LH2[0][2]),
                     (quat2[0], quat2[1], quat2[2], quat2[3]),
                     rospy.Time.now(),
                     "lighthouse2",
                     "world")

    
    #1. COMPUTE VISIBLE SENSORS AT THE MOMENT
    LH1_array = np.asarray(LH1[0])
    LH2_array = np.asarray(LH2[0])
    #testTriangle = np.squeeze(np.asarray(triangles[0]))

    visibleLH1 = 0
    visibleLH2 = 0

    for i, nmbr_sensor in enumerate(sensors):
        sensor = sensor_dict[nmbr_sensor][1]

        #get distance of current sensor and check if intersection
        interDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                             np.squeeze(np.asarray(triangles[nmbr_sensor])), 'lighthouse1')
        interDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                             np.squeeze(np.asarray(triangles[nmbr_sensor])), 'lighthouse2')
        
        #print('interDist');print(interDistLH1);print(interDistLH2);print('endinterDist')

        isVisible1 = True
        isVisible2 = True
        
        for j in range(len(triangles)):
            if(nmbr_sensor != j):
                #print(j)
                if(isVisible1):
                    newInterDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                                        np.squeeze(np.asarray(triangles[j])), 'lighthouse1')
                    if(newInterDistLH1 < interDistLH1 and newInterDistLH1 != False):
                        isVisible1 = False
                        #interDistLH1 = newInterDistLH1
                        #visibleLH1 += 1
                if(isVisible2):
                    newInterDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                                        np.squeeze(np.asarray(triangles[j])), 'lighthouse2')
                    if(newInterDistLH2 < interDistLH2 and newInterDistLH2 != False):
                        isVisible2 = False
                        #interDistLH2 = newInterDistLH2
                        #visibleLH2 += 1
                    
                if(not (isVisible1 or isVisible2)):
                    break
        
        if(isVisible1):
            visibleLH1 += 1
        if(isVisible2):
            visibleLH2 += 1
        #print(newInterDistLH1); print(newInterDistLH2)
        
    #2. COMPUTE EUCLIDEAN DISTANCE OF SENSORS
    individual = sensors
    dist = 0
    for i,ind in enumerate(individual):
        ind = np.asarray(sensor_dict[ind][1])
        for j in range(i,len(individual)):
            if(i != j):
                indivi = np.asarray(sensor_dict[individual[j]][1])
                dist += np.linalg.norm(ind-indivi)

    #print(visibleLH1);print(visibleLH2);print('')
    
    return dist, visibleLH1, visibleLH2

In [17]:
toolbox.register("evaluate", FitnessFunction)

toolbox.register("mate", tools.cxTwoPoint)
# Independent probability  : for each attribute to be mutated.# low~up rondom int
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(sensors.tolist())-1, indpb=0.2) 
toolbox.register("select", tools.selTournament, tournsize=3)

In [18]:
# Creating population

population = toolbox.population(n=10)

In [19]:
hof = tools.HallOfFame(1)

In [20]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [21]:
from data.algorithmsMod import varAnd
from deap import tools
from data.trafomatrix import getRandomRotationmatrix

#MODDED VERSION of eaSimple from DEAP
def eaSimpleMod(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """This algorithm reproduce the simplest evolutionary algorithm as
    presented in chapter 7 of [Back2000]_.

    :param population: A list of individuals.
    :param toolbox: A :class:`~deap.base.Toolbox` that contains the evolution
                    operators.
    :param cxpb: The probability of mating two individuals.
    :param mutpb: The probability of mutating an individual.
    :param ngen: The number of generation.
    :param stats: A :class:`~deap.tools.Statistics` object that is updated
                  inplace, optional.
    :param halloffame: A :class:`~deap.tools.HallOfFame` object that will
                       contain the best individuals, optional.
    :param verbose: Whether or not to log the statistics.
    :returns: The final population
    :returns: A class:`~deap.tools.Logbook` with the statistics of the
              evolution

    The algorithm takes in a population and evolves it in place using the
    :meth:`varAnd` method. It returns the optimized population and a
    :class:`~deap.tools.Logbook` with the statistics of the evolution. The
    logbook will contain the generation number, the number of evalutions for
    each generation and the statistics if a :class:`~deap.tools.Statistics` is
    given as argument. The *cxpb* and *mutpb* arguments are passed to the
    :func:`varAnd` function. The pseudocode goes as follow ::

        evaluate(population)
        for g in range(ngen):
            population = select(population, len(population))
            offspring = varAnd(population, toolbox, cxpb, mutpb)
            evaluate(offspring)
            population = offspring

    As stated in the pseudocode above, the algorithm goes as follow. First, it
    evaluates the individuals with an invalid fitness. Second, it enters the
    generational loop where the selection procedure is applied to entirely
    replace the parental population. The 1:1 replacement ratio of this
    algorithm **requires** the selection procedure to be stochastic and to
    select multiple times the same individual, for example,
    :func:`~deap.tools.selTournament` and :func:`~deap.tools.selRoulette`.
    Third, it applies the :func:`varAnd` function to produce the next
    generation population. Fourth, it evaluates the new individuals and
    compute the statistics on this population. Finally, when *ngen*
    generations are done, the algorithm returns a tuple with the final
    population and a :class:`~deap.tools.Logbook` of the evolution.

    .. note::

        Using a non-stochastic selection method will result in no selection as
        the operator selects *n* individuals from a pool of *n*.

    This function expects the :meth:`toolbox.mate`, :meth:`toolbox.mutate`,
    :meth:`toolbox.select` and :meth:`toolbox.evaluate` aliases to be
    registered in the toolbox.

    .. [Back2000] Back, Fogel and Michalewicz, "Evolutionary Computation 1 :
       Basic Algorithms and Operators", 2000.
    """
    global sensor_dict
    global triangles
    global orientationMesh
    
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print logbook.stream

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Vary the pool of individuals
        offspring = varAnd(offspring, toolbox, cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print logbook.stream
            

        
        if(gen%5==0):
            global sensorDictBackup
            global trianglesBackup
            sensor_dict = sensorDictBackup
            R = getRandomRotationmatrix()
            sensorDictNew = []
            
            for sensor in sensor_dict:
                sensorDictNew.append(np.squeeze(np.asarray(R.dot(np.array(sensor[1])))).tolist())
                
            sensor_dict = list(zip(range(len(sensorDictNew)), sensorDictNew))
        
            tri1 = R.dot(np.transpose(trianglesBackup[:,0:3]))
            tri2 = R.dot(np.transpose(trianglesBackup[:,3:6]))
            tri3 = R.dot(np.transpose(trianglesBackup[:,6:9]))

            triangles = np.concatenate((tri1.T,tri2.T,tri3.T),axis=1)

            orientationMesh = Quaternion(matrix=R)
            
            meshVisualization(orientationMesh, stl_file)
            sensorVisualization(sensor_dict, rate=500, sphereSize=0.03, color=(0,0,1,1))
            
    return population, logbook

In [22]:
population, log = eaSimpleMod(population, 
                                toolbox, 
                                cxpb=0.5, 
                                mutpb=0.2, 
                                ngen=50, 
                                stats=stats, 
                                halloffame=hof, 
                                verbose=True)

gen	nevals	avg    	std    	min	max    
0  	10    	137.987	182.898	6  	406.008
1  	10    	139.486	184.772	5  	406.928
2  	9     	140.528	186.095	7  	406.928
3  	6     	140.769	186.011	7  	406.469
4  	6     	141.457	186.972	7  	407.405
5  	3     	141.461	187.33 	7  	407.99 
6  	8     	141.254	187.676	7  	408.138
7  	5     	141.016	187.353	7  	407.666
8  	5     	140.604	186.769	6  	407.666
9  	8     	140.485	187.582	6  	407.666
10 	4     	139.846	186.636	4  	407.666
11 	8     	140.296	186.894	5  	407.666
12 	6     	140.534	187.305	5  	407.666
13 	6     	140.727	188.277	5  	407.666
14 	3     	141.032	188.001	6  	407.666
15 	9     	140.236	187.8  	7  	407.666
16 	4     	140.113	188.049	7  	407.896
17 	6     	140.13 	188.279	6  	407.896
18 	8     	140.586	188.708	6  	407.896
19 	4     	140.695	188.511	7  	407.896
20 	8     	140.658	188.95 	7  	407.896
21 	7     	141.123	187.991	7  	407.896
22 	5     	141.364	188.398	6  	407.896
23 	7     	141.207	187.825	6  	407.896
24 	5     	141.454	188.03

In [23]:
bestSensors = tools.selBest(population, k=1)[0]
print(bestSensors)

[57, 107, 334, 368, 355, 163, 68, 10, 46, 463, 131, 253, 350, 258, 136, 363, 428, 464, 79, 311, 430, 114, 200, 204, 265]


In [25]:
from data.bestSensorVis import bestSensorVis
#Sensor visualization in RVIZ

bestSensorVis(sensor_dict, bestSensors, rate=500, color=(1,0,0,0.8), sphereSize=0.1)
